In [0]:
# Load the google drive
from google.colab import drive
drive.mount('/content/gdrive')
directory = '/content/gdrive/My Drive/ColorNetData/'
train_directory = directory + 'Train/'
test_directory = directory + 'Test/'

# Load all the required data
from pickle import load
train_image_encoding = load(open(directory + 'train_image_encoding.p', 'rb'))
test_image_encoding = load(open(directory + 'test_image_encoding.p', 'rb'))
model = load(open(directory + 'final_model.p', 'rb'))
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.load_weights(directory + 'weights.h5')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Define the data generator
def data_generator(batch_size=32):
  import numpy as np
  from skimage.color import rgb2lab, lab2rgb
  from keras.preprocessing.image import load_img, img_to_array
  encoder_input = list()
  embedding_input = list()
  model_output = list()
  from itertools import cycle
  for name, embedding in cycle(train_image_encoding.items()):
    # Encoded image input
    image = np.array(img_to_array(load_img(train_directory + name, target_size=(256, 256))), dtype=float)
    encoded_image = rgb2lab(1.0/255*image)[:, :, 0]
    encoded_image = encoded_image.reshape(encoded_image.shape[0], encoded_image.shape[1], 1)
    # Output image
    output_image = rgb2lab(1.0/255*image)[:, :, 1:]
    output_image = output_image/128
    output_image = output_image.reshape(output_image.shape[0], output_image.shape[1], 2)
    # Add to return
    encoder_input.append(encoded_image)
    embedding_input.append(embedding.flatten())
    model_output.append(output_image)
    
    if len(model_output) >= batch_size:
      encoder_input = np.asarray(encoder_input)
      embedding_input = np.asarray(embedding_input)
      model_output = np.asarray(model_output)
      yield [[encoder_input, embedding_input], model_output]
      encoder_input = list()
      embedding_input = list()
      model_output = list()

In [0]:
# Execute this to train
batch_size = 32
nb_epoch = 1
steps_per_epoch = int(len(train_image_encoding)/batch_size)
model.fit_generator(data_generator(batch_size), nb_epoch=nb_epoch, verbose=1, steps_per_epoch=steps_per_epoch)
model.save_weights(directory + 'weights.h5')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=1, steps_per_epoch=262, epochs=1)`
  after removing the cwd from sys.path.


Epoch 1/1
  2/262 [..............................] - ETA: 55:45 - loss: 0.0122 - mean_squared_error: 0.0122  